In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Loading csv data
bids = pd.read_csv('./datas/bids.csv')

In [4]:
# 1. Get the number of unique IPs per auction
auctionIpCounts = bids.groupby('auction')['ip'].nunique().reset_index()
print(auctionIpCounts)

      auction    ip
0       00270   771
1       008vv     2
2       00cwr    13
3       00do0     2
4       00hjy    32
...       ...   ...
15046   zznnc     2
15047   zzoy7   196
15048   zztha     3
15049   zzzlx  3519
15050   zzzy7  2085

[15051 rows x 2 columns]


In [5]:
# 2. Count max bids per merchandise per country
groupedMerchadiseByCountry = bids.groupby(['merchandise', 'country']).size().reset_index(name='bid_count')
max_bids_per_merchandise = groupedMerchadiseByCountry.loc[groupedMerchadiseByCountry.groupby('merchandise')['bid_count'].idxmax()]
print(max_bids_per_merchandise)

           merchandise country  bid_count
14          auto parts      in       2243
101    books and music      in      11820
230           clothing      in       7619
411          computers      th      16731
495          furniture      in      47031
675         home goods      in     169649
861            jewelry      in     560773
1050            mobile      in     578076
1225  office equipment      in      56140
1411    sporting goods      in     346852


In [8]:
# 3. Calculate average number of unique bidders per auction in each of the three separate time periods
# Convert the time column to datetime
bids['time'] = pd.to_datetime(bids['time'])
start_time = bids['time'].min()
end_time = bids['time'].max()
periods = pd.date_range(start=start_time, end=end_time, periods=4)
# Attribute a period to each record
bids['period'] = pd.cut(bids['time'], bins=periods, labels=['Period 1', 'Period 2', 'Period 3'])
# Group by period and auction, then find unique bidders for each auction
grouped = bids.groupby(['period', 'auction'])['bidder_id'].nunique().reset_index(name='unique_bidders')
# Calculate the average number of unique bidders per auction for each period
average_unique_bidders = grouped.groupby('period')['unique_bidders'].mean().reset_index(name='average_unique_bidders')
print(average_unique_bidders)

/tmp/ipykernel_6300/2461763805.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = bids.groupby(['period', 'auction'])['bidder_id'].nunique().reset_index(name='unique_bidders')


     period  average_unique_bidders
0  Period 1                9.409807
1  Period 2                6.611720
2  Period 3                6.127500
3  Period 4                9.177463


/tmp/ipykernel_6300/2461763805.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_unique_bidders = grouped.groupby('period')['unique_bidders'].mean().reset_index(name='average_unique_bidders')
